In [1]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import os
import time
import warnings
import gc
import os
from six.moves import urllib
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
warnings.filterwarnings('ignore')
%matplotlib inline
plt.style.use('seaborn')

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
history = pd.read_csv('./data/historical_transactions.csv')
new = pd.read_csv('./data/new_merchant_transactions.csv')

In [6]:
df = pd.concat([history,new])

In [7]:
df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [9]:
#category_3이 A일 때 installments의 값
set(df.loc[df['category_3']=='A','installments'])

{0}

In [11]:
#category_3이 B일 때 installments의 값
set(df.loc[df['category_3']=='B','installments'])

{1}

In [13]:
#category_3이 C일 때 installments의 값
set(df.loc[df['category_3']=='C','installments'])

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 999}

In [14]:
#category_3이 NaN일 때 installments의 값
set(df.loc[df['category_3'].isnull(),'installments'])

{-1}

### category_3는 할부여부와 관련되었다고 예상할 수 있음
- 0 = 직불카드
- 1 = 신용카드(일시불)
- 2~12 = 신용카드(할부)
- 999 = 190개, 이상치로 처리
- -1 = ?

In [15]:
#category_2가 1일 때 state_id의 값
set(df.loc[df['category_2']==1,'state_id'])

{9, 12, 15, 16}

In [16]:
#category_2가 2일 때 state_id의 값
set(df.loc[df['category_2']==2,'state_id'])

{6, 10, 18, 23, 24}

In [17]:
#category_2가 3일 때 state_id의 값
set(df.loc[df['category_2']==3,'state_id'])

{1, 2, 3, 7, 8, 11, 17, 19}

In [18]:
#category_2가 4일 때 state_id의 값
set(df.loc[df['category_2']==4,'state_id'])

{4, 13, 14, 22}

In [19]:
#category_2가 5일 때 state_id의 값
set(df.loc[df['category_2']==5,'state_id'])

{5, 20, 21}

In [20]:
#category_2가 NaN일 때 state_id의 값
set(df.loc[df['category_2'].isnull(),'state_id'])

{-1}

### category_2는 지역과 관련되었다고 예상할 수 있음
- -1은 온라인으로 예상

In [35]:
#category_1이 Y일 때 city_id의 값
set(df.loc[df['category_1']=='Y','city_id'])

{-1}

In [37]:
#category_1이 Y일 때 state_id의 값
set(df.loc[df['category_1']=='Y','state_id'])

{-1}

In [36]:
#category_1이 N일 때 city_id의 값
set(df.loc[df['category_1']=='N','city_id'])

{-1,
 1,
 2,
 3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 28,
 29,
 30,
 32,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 44,
 46,
 47,
 48,
 49,
 51,
 52,
 53,
 54,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 122,
 123,
 124,
 125,
 126,
 128,
 129,
 130,
 131,
 133,
 135,
 136,
 137,
 138,
 139,
 140,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 153,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 193,
 194,
 195,
 197,
 198,
 199,
 200,
 201,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 

In [38]:
#category_1이 Y일 때 state_id의 값
set(df.loc[df['category_1']=='N','state_id'])

{-1,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24}

In [23]:
set(df.loc[df['city_id']== -1,'state_id'])

{-1}

In [29]:
set(df.loc[df['category_1']== 'Y','state_id'])

{-1}

In [34]:
len(df.loc[(df['category_1']== 'N') & (df['state_id'] ==- -1)])

265661

### category_1는 해외/국내 여부와 관련되었다고 예상할 수 있음
- category_1이 Y면 city와 state는 모두 -1이지만 N일 때도 -1값이 있긴 함